In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
# Construct a BigQuery client object.
#client = bigquery.Client()
# parse the config json file to the client
client = bigquery.Client.from_service_account_json("/path/to/bq_cred.json")

In [ ]:
# define function to execute query
def execute_bq_query(sql_file_name):
        # Read in query from text file
    with open(sql_file_name, 'r') as file:
        query = file.read()

    # Execute the query
    query_job = client.query(query)

    # Process the results
    results = query_job.result()

    print(results)
    
    # Convert results to dataframe
    # use the field names as the column names
    df = pd.DataFrame(data=[list(row.values()) for row in results],
                  columns=[field.name for field in results.schema])
    return df

In [ ]:
df = execute_bq_query('get_term_courses.sql')
# print number of rows in the dataframe
print(df.shape[0])

In [ ]:
# print dataframe columns
print(df.columns)

In [ ]:
from canvasapi import Canvas
from dotenv import load_dotenv
import os

#Load environment file for secrets.
try:
    if load_dotenv('.env') is False:
        raise TypeError
except TypeError:
    print('Unable to load .env file.')
    quit()

# init canvas api instance
API_URL = os.getenv('API_URL')
API_KEY = os.getenv('API_KEY')
canvas = Canvas(API_URL, API_KEY)

In [ ]:
# loop through the dataframe and print the first 5 rows
for index, row in df.iterrows():
    course_id = row['course_id']
    print(course_id)
    # add tool into the course navigation
    course = canvas.get_course(course_id)
    course_tabs = course.get_tabs()
    for tab in course_tabs:
        if tab.label == 'Instructor Tools - Test':
            print('found the tab')
            # update its position
            tab.update(position=2)
    break